In [53]:
import numpy as np
from scipy.stats import unitary_group

np.set_printoptions(precision=3, suppress=True)

In [89]:
def M(j, k, sigma, delta, m):
    r"""The Bell M matrix from Eq 1 of the paper"""
    mat = np.identity(m, dtype=np.complex128)
    n = j - k
    mat[n, n] = np.exp(1j * sigma) * np.sin(delta)
    mat[n, n+1] = np.exp(1j * sigma) * np.cos(delta)
    mat[n+1, n] = np.exp(1j * sigma) * np.cos(delta)
    mat[n+1, n+1] = -np.exp(1j * sigma) * np.sin(delta)
    return mat

def P(j, phi, m):
    mat = np.identity(m, dtype=np.complex128)
    mat[j,j] = np.exp(1j * phi)
    return mat

In [90]:
def is_unitary(U):
    return np.allclose(U @ U.conj().T, np.eye(U.shape[0]))

In [91]:
m = 4
# V = np.random.random((5,5)) + 1j*np.random.random((5,5))
U = unitary_group.rvs(m)

V = U.conj()

In [96]:
for j in range(m-1):
    x = m - 1
    y = j
    phi_j = - np.angle(V[x, y]) + np.angle(V[x, y+1])
    Pj = P(j, phi_j, m)
    V = V @ Pj
    print(np.angle(V[x,y]), np.angle(V[x, y+1]))
    for k in range(j+1):
        delta = np.arctan(-abs(V[x,y+1]) / abs(V[x,y]))
        print(delta)
        V_temp = V @ M(j, k, 0, delta, m)
        sigma = np.angle(V_temp[x-1, y-1]) - np.angle(V_temp[x-1,y])
        V = V @ M(j, k, sigma, delta, m)
        x -= 1
        y -= 1
    
zeta = - np.angle(V[0,0])
V = V @ P(0, zeta, m)
for j in range(1,m):
    zeta = np.angle(V[0,0]) - np.angle(V[j,j])
    V = V @ P(j, zeta, m)
        
V

2.0664025842695577 2.0664025842695577
-0.18358933978032815
-0.6087970719546929 -0.608797071954693
-0.07611284015551614
-1.5707963267948966
2.8336775009728457 2.8336775009728457
-1.5707963267948966
-0.07611284015551609
-0.1835893397803282


array([[ 1.-0.j, -0.+0.j, -0.-0.j,  0.+0.j],
       [ 0.+0.j,  1.+0.j, -0.+0.j,  0.-0.j],
       [-0.+0.j, -0.+0.j,  1.+0.j, -0.+0.j],
       [-0.-0.j,  0.-0.j, -0.+0.j,  1.+0.j]])

In [87]:
for j in range(m-1):
    x = m - 1
    y = j
    phi_j = np.angle(V[x, y]) - np.angle(V[x, y+1])
    Pj = P(j, phi_j, m)
    V = V @ Pj
    for k in range(j+1):
        delta = np.arctan(-V[x,y+1]/V[x,y])
        if k==j: 
            sigma = 0 #doesnt matter with the result
        else:
            sigma = np.arctan(V[x-1,y]/V[x-1,y-1]) #Working on this factor
        M = np.eye(m,dtype=np.complex128)
        if j ==k:
            M[0,0] = np.exp(1j*sigma)*np.sin(delta)
            M[0,1] = np.exp(1j*sigma)*np.cos(delta)
            M[1,0] = np.exp(1j*sigma)*np.cos(delta)
            M[1,1] = -np.exp(1j*sigma)*np.sin(delta)
        elif j==k+1:
            M[1,1] = np.exp(1j*sigma)*np.sin(delta)
            M[1,2] = np.exp(1j*sigma)*np.cos(delta)
            M[2,1] = np.exp(1j*sigma)*np.cos(delta)
            M[2,2] = -np.exp(1j*sigma)*np.sin(delta)

        V=V@M
        x = x-1
        y = y-1
        print(V)

[[-0.439-0.855j -0.518-0.084j -1.552-0.381j -0.393+0.265j]
 [-1.285+0.946j -0.509+0.373j -1.503+2.066j -1.158+0.436j]
 [-0.   -0.j     0.   -0.j    -2.133+1.569j -0.86 +0.705j]
 [ 0.   -0.j    -0.   +0.j     0.   +0.j    -0.444+0.326j]]
[[-0.439-0.855j -1.544+0.755j  0.033-0.125j -0.393+0.265j]
 [-1.285+0.946j  0.286+2.673j -0.109-0.509j -1.158+0.436j]
 [-0.   -0.j    -0.407+2.906j  0.777-0.517j -0.86 +0.705j]
 [ 0.   -0.j    -0.   +0.j    -0.   +0.j    -0.444+0.326j]]
[[-0.439-0.855j  1.544-0.755j  0.033-0.125j -0.393+0.265j]
 [-1.285+0.946j -0.286-2.673j -0.109-0.509j -1.158+0.436j]
 [-0.   +0.j     0.407-2.906j  0.777-0.517j -0.86 +0.705j]
 [ 0.   -0.j     0.   -0.j    -0.   +0.j    -0.444+0.326j]]
[[-0.439-0.855j  1.544-0.755j  0.006-0.129j -0.393+0.265j]
 [-1.285+0.946j -0.286-2.673j -0.211-0.475j -1.158+0.436j]
 [-0.   +0.j     0.407-2.906j  0.654-0.666j -0.86 +0.705j]
 [ 0.   -0.j     0.   -0.j    -0.   +0.j    -0.444+0.326j]]
[[-0.439-0.855j -0.231+0.258j -0.145-1.041j -0.393+0

In [47]:
V

array([[ 0.766+0.053j, -0.724-1.69j ,  1.773-1.086j,  0.096+0.639j],
       [ 0.451+2.112j,  2.682-1.985j,  0.172-1.361j,  0.209-0.095j],
       [-0.   -0.j   , -1.651-0.138j,  2.365-0.948j, -0.234-0.105j],
       [-0.   -0.j   ,  0.   +0.j   ,  1.299+1.426j,  0.679-0.051j]])